#Importing Libraries

In [4]:
pip install pulp

In [5]:
import numpy as np
import pandas as pd
from pulp import *
import re
from scipy.stats import binom

##Loading the Dataset

In [6]:
df_ipl = pd.read_csv("/content/IPL-aggregated-stats_final.csv")

In [7]:
df_ipl.head()

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,6s_mean,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,overseas_or_indian,median_season_ratings,avg_season_ratings,base_price
0,ABdeVilliers,1712.5,155.681818,11,139,12.636364,0,0.000000,0,0.000000,...,18.181818,80,7.272727,7,0.636364,BAT,Overseas,50.344828,51.410021,1.50
1,AaronFinch,751.0,83.444444,9,75,8.333333,1,0.111111,13,1.444444,...,7.444444,24,2.666667,0,0.000000,BAT,Overseas,11.699010,27.457438,0.75
2,AbhishekJhunjhunwala,120.5,40.166667,3,21,7.000000,1,0.333333,27,9.000000,...,1.666667,10,3.333333,0,0.000000,BAT,Indian,2.472464,4.623134,0.30
3,AbhishekNayar,340.5,48.642857,7,60,8.571429,9,1.285714,69,9.857143,...,2.857143,13,1.857143,0,0.000000,BAT,Indian,2.480645,8.571047,0.30
4,AdamGilchrist,1091.5,181.916667,6,80,13.333333,1,0.166667,1,0.166667,...,15.333333,51,8.500000,16,2.666667,WK,Overseas,70.380747,61.764019,2.00


##Choosen List of players

In [8]:
Choosen_list = ["DavidWarner", "AxarPatel", "AdamGilchrist", "NitishRana", "MohitSharma", "BhuvneshwarKumar",
                 "MitchellMcClenaghan", "AnilKumble", "AzharMahmood", "YuvrajSingh", "PraveenKumar"]
sum(df_ipl[df_ipl.PLAYER.isin(Choosen_list)]["median_season_ratings"])

742.3838527243583

##Count of overseas and indian players

In [9]:
df_ipl.overseas_or_indian.value_counts()

overseas_or_indian
Indian      135
Overseas    104
Name: count, dtype: int64

### The objective function and accompanying constraints are described below:

Each team can only select 11 players.

Batsman - Min 3 Max 5

Bowlers - Min 3 Max 5

All Rounders - Min 1 Max 3

Wicketkeepers - 1

Max Spending limit - 30 crores. 7 . Overseas Players - 4

max \sum_{players =1}^N rating

min-batsman <  chosen-batsman < max-batsman

 min-batsman <  chosen-batsman < max-batsman

min-bowlers <  chosen-bowlers < max-bowlers

min-allrounders <  chosen-all-rounders < max-allrounders

wicketkeepers = 1

### Forming the group of players to be included in the auction.

In [10]:
players = list(df_ipl["PLAYER"])
indians = list(df_ipl[df_ipl["overseas_or_indian"]=="Indian"]["PLAYER"])
overseas = list(df_ipl[df_ipl["overseas_or_indian"]=="Overseas"]["PLAYER"])
batsman = list(df_ipl[df_ipl.position=="BAT"]["PLAYER"])
bowlers = list(df_ipl[df_ipl.position=="BWL"]["PLAYER"])
allrounders = list(df_ipl[df_ipl.position=="AR"]["PLAYER"])
wks = list(df_ipl[df_ipl.position=="WK"]["PLAYER"])

In [11]:
df_ipl[df_ipl.PLAYER.isin(["PraveenKumar", "ZaheerKhan", "SiddarthKaul"])]

,PLAYER,Pts_sum,Pts_mean,years_played,Mat_sum,Mat_mean,Wkts_sum,Wkts_mean,Dots_sum,Dots_mean,...,6s_mean,Catches_sum,Catches_mean,Stumpings_sum,Stumpings_mean,position,overseas_or_indian,median_season_ratings,avg_season_ratings,base_price
160,PraveenKumar,1538.0,153.80,10,119,11.9,90,9.0,1076,107.6,...,1.7,12,1.2,0,0.0,BWL,Indian,70.410509,53.318622,2.00
205,SiddarthKaul,509.0,101.80,5,45,9.0,49,9.8,320,64.0,...,0.0,7,1.4,0,0.0,BWL,Indian,8.929870,28.434179,0.75
238,ZaheerKhan,1321.5,132.15,10,100,10.0,102,10.2,875,87.5,...,0.2,22,2.2,0,0.0,BWL,Indian,39.205645,42.608871,1.00


In [12]:
median_ratings = dict(zip(players, list(df_ipl["median_season_ratings"])))
base_prices = dict(zip(players, list(df_ipl["base_price"])))

### Let's implement a strategy using Binary Integer Programming

In [13]:
Lp_players = LpVariable.dicts("Chosen",players,0,1,cat='Integer')

In [14]:
def Lp_filter_dict(Lp_players, criteria_list):
    dict_filter = {}
    for player in criteria_list:
        dict_filter[player] = Lp_players[player]
    return dict_filter

In [15]:
batsman_lp = Lp_filter_dict(Lp_players, batsman)
bowlers_lp = Lp_filter_dict(Lp_players, bowlers)
ar_lp = Lp_filter_dict(Lp_players, allrounders)
wk_lp = Lp_filter_dict(Lp_players, wks)
indians_lp = Lp_filter_dict(Lp_players, indians)
overseas_lp = Lp_filter_dict(Lp_players, overseas)

In [16]:
def Lp_construct_SumArr(dict_lp):
    arr_lp = []
    for k,v in dict_lp.items():
        arr_lp += lpSum(dict_lp[k])
    return arr_lp

In [17]:
prob = LpProblem("IPL_Auction", LpMaximize)

In [18]:
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in Lp_players.items():
    costs += lpSum(base_prices[k] * Lp_players[k])
    rewards += lpSum(median_ratings[k] * Lp_players[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(Lp_construct_SumArr(wk_lp)) == 1
prob += lpSum(Lp_construct_SumArr(Lp_players)) == 11
prob += lpSum(Lp_construct_SumArr(batsman_lp)) >= 3
prob += lpSum(Lp_construct_SumArr(batsman_lp)) <= 5
prob += lpSum(Lp_construct_SumArr(bowlers_lp)) >= 3
prob += lpSum(Lp_construct_SumArr(bowlers_lp))<= 5
prob += lpSum(Lp_construct_SumArr(ar_lp)) >= 1
prob += lpSum(Lp_construct_SumArr(ar_lp)) <= 3
prob += lpSum(Lp_construct_SumArr(indians_lp)) == 7
prob += lpSum(Lp_construct_SumArr(overseas_lp)) == 4

In [19]:
prob.solve()

1

In [20]:
prob.writeLP("/content/IPL_Best11_Rounds-5-Inflation.lp")

[Chosen_ABdeVilliers,
 Chosen_AaronFinch,
 Chosen_AbhishekJhunjhunwala,
 Chosen_AbhishekNayar,
 Chosen_AdamGilchrist,
 Chosen_AjanthaMendis,
 Chosen_AjinkyaRahane,
 Chosen_AjitAgarkar,
 Chosen_AkshdeepNath,
 Chosen_AlbieMorkel,
 Chosen_AmbatiRayudu,
 Chosen_AmitMishra,
 Chosen_AndreRussell,
 Chosen_AndrewSymonds,
 Chosen_AndrewTye,
 Chosen_AngeloMathews,
 Chosen_AnilKumble,
 Chosen_AnkitRajpoot,
 Chosen_AnkitSharma,
 Chosen_AnureetSingh,
 Chosen_AshishNehra,
 Chosen_AshishReddy,
 Chosen_AshokDinda,
 Chosen_AxarPatel,
 Chosen_AzharMahmood,
 Chosen_BarinderSran,
 Chosen_BenCutting,
 Chosen_BenStokes,
 Chosen_BharatChipli,
 Chosen_BhuvneshwarKumar,
 Chosen_BipulSharma,
 Chosen_BradHodge,
 Chosen_BradHogg,
 Chosen_BrendonMcCullum,
 Chosen_BrettLee,
 Chosen_CMGautam,
 Chosen_CameronWhite,
 Chosen_ChamindaVaas,
 Chosen_CharlLangeveldt,
 Chosen_CheteshwarPujara,
 Chosen_ChrisGayle,
 Chosen_ChrisJordan,
 Chosen_ChrisLynn,
 Chosen_ChrisMorris,
 Chosen_ColinMunro,
 Chosen_ColindeGrandhomme,
 Cho

In [21]:
def summary(prob):
    div = '---------------------------------------\n'
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Constraints:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))

In [22]:
summary(prob)

Variables:

Chosen_AnilKumble = 1.0
Chosen_BhuvneshwarKumar = 1.0
Chosen_ChrisGayle = 1.0
Chosen_DavidWarner = 1.0
Chosen_HarbhajanSingh = 1.0
Chosen_JacquesKallis = 1.0
Chosen_MunafPatel = 1.0
Chosen_RashidKhan = 1.0
Chosen_RishabhPant = 1.0
Chosen_SureshRaina = 1.0
Chosen_VirenderSehwag = 1.0
---------------------------------------

Constraints:
2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 + 2.0*1.0 = 22.0
---------------------------------------

Score:
81.0*1.0 + 80.66666666666667*1.0 + 80.47619047619048*1.0 + 80.55555555555556*1.0 + 80.49107142857143*1.0 + 70.33333333333333*1.0 + 80.38461538461539*1.0 + 81.66666666666667*1.0 + 75.96428571428572*1.0 + 80.51315789473685*1.0 + 80.58823529411765*1.0 = 872.6397784147397


### Clearly, players won't be sold at their base price. We should use an inflated price to forecast their final sale price

In [23]:
def calculate_bidding_increment(price):
    if price <= 0.75:
        return 0.1
    elif (price >= 0.75 ) and (price < 1.5):
        return 0.2
    elif (price >= 1.5) and (price < 3.5):
        return 0.3
    elif (price >= 3.5) and (price < 5):
        return 0.5
    else:
        return 1

In [24]:
def price_post_betting(base_price, rounds):
    price = base_price
    while(rounds > 0):
        price += calculate_bidding_increment(price)
        price = round(price, 1)
        rounds-=1
    return price

In [25]:
dict_inv_base_price= {2: ">90", 1.5: "80-90", 1: "60-80", 0.75:"50-60", 0.5:"40-50", 0.4:"30-40", 0.3:"<30"}

In [26]:
def simulate_price(base_price, n=20):
    dict_inv_base_price = {2: ">90", 1.5: "80-90", 1: "60-80", 0.75:"50-60", 0.5:"40-50", 0.4:"30-40", 0.3:"<30"}
    prob_dist = {
    ">90": [binom.pmf(i, n, 0.4) for i in range(n+1)],
    "80-90": [binom.pmf(i, n, 0.3) for i in range(n+1)],
    "60-80": [binom.pmf(i, n, 0.2) for i in range(n+1)],
    "50-60": [binom.pmf(i, n, 0.15) for i in range(n+1)],
    "40-50": [binom.pmf(i, n, 0.15) for i in range(n+1)],
    "30-40": [binom.pmf(i, n, 0.12) for i in range(n+1)],
    "<30": [binom.pmf(i, n, 0.1) for i in range(n+1)]
    }
    prices = [price_post_betting(base_price, i) for i in range(n+1)]
    prob = prob_dist[dict_inv_base_price[base_price]]
    simulated_mean = sum([prices[i] * prob[i] for i in range(n+1)])
    return simulated_mean if simulated_mean > base_price else base_price

### We assume that all players go through at least five rounds of bidding, where their price is increased five times.

In [27]:
inflated_price = dict(zip(players, list(df_ipl["base_price"].apply(lambda x : price_post_betting(x, 5)))))

In [28]:
df_ipl.base_price = df_ipl.base_price.astype(float)

In [29]:
df_ipl["simulated_price"] = list(df_ipl["base_price"].apply(lambda x : simulate_price(x)))


In [30]:
simulated_price = dict(zip(players, df_ipl["simulated_price"]))

In [31]:
prob = LpProblem("IPL_Auction", LpMaximize)
rewards = []
costs = []
MAX_AVAILABLE_COST = 30
for k, v in Lp_players.items():
    costs += lpSum(simulated_price[k] * Lp_players[k])
    rewards += lpSum(median_ratings[k] * Lp_players[k])
prob+= lpSum(rewards)
prob += lpSum(costs) <= MAX_AVAILABLE_COST
prob += lpSum(Lp_construct_SumArr(wk_lp)) == 1
prob += lpSum(Lp_construct_SumArr(Lp_players)) == 11
prob += lpSum(Lp_construct_SumArr(batsman_lp)) >= 3
prob += lpSum(Lp_construct_SumArr(batsman_lp)) <= 5
prob += lpSum(Lp_construct_SumArr(bowlers_lp)) >= 3
prob += lpSum(Lp_construct_SumArr(bowlers_lp))<= 5
prob += lpSum(Lp_construct_SumArr(ar_lp)) >= 1
prob += lpSum(Lp_construct_SumArr(ar_lp)) <= 3
prob += lpSum(Lp_construct_SumArr(indians_lp)) == 7
prob += lpSum(Lp_construct_SumArr(overseas_lp)) == 4

In [32]:
prob.solve()

1

In [33]:
summary(prob)

Variables:

Chosen_AjinkyaRahane = 1.0
Chosen_AxarPatel = 1.0
Chosen_BrendonMcCullum = 1.0
Chosen_DavidWarner = 1.0
Chosen_IshantSharma = 1.0
Chosen_MuttiahMuralitharan = 1.0
Chosen_NitishRana = 1.0
Chosen_RashidKhan = 1.0
Chosen_SachinTendulkar = 1.0
Chosen_SiddharthTrivedi = 1.0
Chosen_YuvrajSingh = 1.0
---------------------------------------

Constraints:
1.9288164114295785*1.0 + 3.3945146964406856*1.0 + 1.9288164114295785*1.0 + 5.445510930412082*1.0 + 1.9288164114295785*1.0 + 1.9288164114295785*1.0 + 1.9288164114295785*1.0 + 5.445510930412082*1.0 + 1.9288164114295785*1.0 + 1.9288164114295785*1.0 + 1.9288164114295785*1.0 = 29.716067848701485
---------------------------------------

Score:
50.23255813953488*1.0 + 62.712500000000006*1.0 + 44.22727272727273*1.0 + 80.55555555555556*1.0 + 50.23255813953488*1.0 + 50.208333333333336*1.0 + 44.20842379504994*1.0 + 81.66666666666667*1.0 + 41.25764895330113*1.0 + 44.22704081632653*1.0 + 44.19230769230769*1.0 = 593.7208658188833


In [34]:
prob.writeLP("/content/IPL_Best11_Rounds-5-Inflation.lp")

[Chosen_ABdeVilliers,
 Chosen_AaronFinch,
 Chosen_AbhishekJhunjhunwala,
 Chosen_AbhishekNayar,
 Chosen_AdamGilchrist,
 Chosen_AjanthaMendis,
 Chosen_AjinkyaRahane,
 Chosen_AjitAgarkar,
 Chosen_AkshdeepNath,
 Chosen_AlbieMorkel,
 Chosen_AmbatiRayudu,
 Chosen_AmitMishra,
 Chosen_AndreRussell,
 Chosen_AndrewSymonds,
 Chosen_AndrewTye,
 Chosen_AngeloMathews,
 Chosen_AnilKumble,
 Chosen_AnkitRajpoot,
 Chosen_AnkitSharma,
 Chosen_AnureetSingh,
 Chosen_AshishNehra,
 Chosen_AshishReddy,
 Chosen_AshokDinda,
 Chosen_AxarPatel,
 Chosen_AzharMahmood,
 Chosen_BarinderSran,
 Chosen_BenCutting,
 Chosen_BenStokes,
 Chosen_BharatChipli,
 Chosen_BhuvneshwarKumar,
 Chosen_BipulSharma,
 Chosen_BradHodge,
 Chosen_BradHogg,
 Chosen_BrendonMcCullum,
 Chosen_BrettLee,
 Chosen_CMGautam,
 Chosen_CameronWhite,
 Chosen_ChamindaVaas,
 Chosen_CharlLangeveldt,
 Chosen_CheteshwarPujara,
 Chosen_ChrisGayle,
 Chosen_ChrisJordan,
 Chosen_ChrisLynn,
 Chosen_ChrisMorris,
 Chosen_ColinMunro,
 Chosen_ColindeGrandhomme,
 Cho